In [346]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn import preprocessing

%matplotlib inline
#Score 0.78468

In [347]:
def sigmoid(z):
    """
    Need doctring
    """
    
    # create a placeholder for x
    x = tf.placeholder(tf.float32, name='x')
        
    with tf.Session() as sess:
        result = sess.run(tf.sigmoid(x), feed_dict = {x : z})
    
    return result

def test_sigmoid():
    pass

In [348]:
def get_cost(logits, labels):
    """"""
    y = tf.placeholder(tf.float32, name='y')
    z = tf.placeholder(tf.float32, name='z')
    
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)
    
    with tf.Session() as sess:
        cost = sess.run(cost, feed_dict={z: logits, y: labels})
    
    return cost

In [349]:
# import train and test data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


X_train = train_df
Y_train = train_df
# X_train = train_df.drop("Survived", axis=1)
# Y_train = train_df["Survived"]
# X_train.shape

X_test = test_df
test_passenger_id= test_df["PassengerId"]


# determine data shape and configuration
print('no. of training examples: ' + str(X_train.shape[1]))
print('shape of X_train: ' + str(X_train.shape))
print('shape of Y_train: ' + str(Y_train.shape))

X_train.head(10)
X_test.head(10)


no. of training examples: 12
shape of X_train: (891, 12)
shape of Y_train: (891, 12)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [350]:
# helper functions to preprocess
def drop_cols(dframe, cols):
    return data.drop(cols, axis=1)

cols_to_drop = ['PassengerId','Name', 'Ticket', 'Cabin','Embarked']
X_train = X_train.drop(cols_to_drop, axis=1)
X_test = X_test.drop(cols_to_drop, axis=1)

# find columns that contain NaN values if any
X_train.columns[X_train.isna().any()].tolist()
X_train.columns[X_train.isnull().any()].tolist()
# find number of NaN values for each column
X_train.isna().sum()

# X_train['Fare'].values



Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
dtype: int64

In [351]:
# some sklearn preprocessing 
from sklearn.preprocessing import LabelEncoder

def encode_label(data, col):
    le = LabelEncoder()
    le.fit(data[col].unique().tolist())
    data[col]=le.transform(data[col]) 
    return data
    
# X_train['Sex'].unique().tolist()

X_train = encode_label(X_train, 'Sex')
X_test = encode_label(X_test, 'Sex')
X_train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
5,0,3,1,NaN,0,0,8.4583
6,0,1,1,54.0,0,0,51.8625
7,0,3,1,2.0,3,1,21.0750
8,1,3,0,27.0,0,2,11.1333
9,1,2,0,14.0,1,0,30.0708


In [352]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, [n_x, None], name='X')
    Y = tf.placeholder(tf.float32, [n_y, None], name='Y')
    
    return X, Y

In [353]:
from sklearn.preprocessing import Imputer

def nan_padding(data, columns):
    for column in columns:
        imputer=Imputer()
        data[column]=imputer.fit_transform(data[column].values.reshape(-1,1))
    return data


nan_columns = ["Age", "SibSp", "Parch"]

X_train = nan_padding(X_train, nan_columns)
X_test = nan_padding(X_test, nan_columns)

def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix=column)], axis=1)
        data = data.drop(column, axis=1)
    return data


X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].median())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].median())

# dummy_columns = ["Pclass"]
# X_train=dummy_data(X_train, dummy_columns)
# X_test=dummy_data(X_test, dummy_columns)
X_train.head(10)

X_train.isna().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
dtype: bool

In [354]:

# for dataset in [X_train, X_test]:
#     # Mapping Embarked
# #     dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
#     # Mapping Fare
#     dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
#     dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
#     dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
#     dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
#     dataset['Fare'] = dataset['Fare'].astype(int)
    
#     # Mapping Age
#     dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
#     dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
#     dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
#     dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4

In [355]:
# def initialize_params():
#     """ """
#     W1 = tf.get_variable('W1', )
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer


def normalize_age(data):
    scaler = MinMaxScaler()
    data["Age"] = scaler.fit_transform(data["Age"].values.reshape(-1,1))
    return data
X_train = normalize_age(X_train)
X_test = normalize_age(X_test)
X_train.head()

# train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train)

def split_valid_test_data(data, fraction=(1 - 0.8)):
    data_y = data["Survived"]
    lb = LabelBinarizer()
    data_y = lb.fit_transform(data_y)

    data_x = data.drop(["Survived"], axis=1)

    train_x, valid_x, train_y, valid_y = train_test_split(data_x, data_y, test_size=fraction)

    return train_x.values, train_y, valid_x, valid_y
train_x, train_y, valid_x, valid_y = split_valid_test_data(X_train)


print("train_x:{}".format(train_x.shape))
print("train_y:{}".format(train_y.shape))
print("train_y content:{}".format(train_y[:3]))

print("valid_x:{}".format(valid_x.shape))
print("valid_y:{}".format(valid_y.shape))



# train_x.head(10)

train_x:(712, 6)
train_y:(712, 1)
train_y content:[[1]
 [1]
 [0]]
valid_x:(179, 6)
valid_y:(179, 1)


In [356]:
from collections import namedtuple

def build_neural_network(hidden_units=10):
    tf.reset_default_graph()
    inputs = tf.placeholder(tf.float32, shape=[None, train_x.shape[1]])
    labels = tf.placeholder(tf.float32, shape=[None, 1])
    learning_rate = tf.placeholder(tf.float32)
    is_training=tf.Variable(True,dtype=tf.bool)
    
    initializer = tf.contrib.layers.xavier_initializer()
    fc = tf.layers.dense(inputs, hidden_units, activation=None,kernel_initializer=initializer)
    fc=tf.layers.batch_normalization(fc, training=is_training)
    fc=tf.nn.relu(fc)
    
    logits = tf.layers.dense(fc, 1, activation=None)
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
    cost = tf.reduce_mean(cross_entropy)
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    predicted = tf.nn.sigmoid(logits)
    correct_pred = tf.equal(tf.round(predicted), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Export the nodes 
    export_nodes = ['inputs', 'labels', 'learning_rate','is_training', 'logits',
                    'cost', 'optimizer', 'predicted', 'accuracy']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])

    return graph

model = build_neural_network()

In [357]:
def get_batch(data_x,data_y,batch_size=32):
    batch_n=len(data_x)//batch_size
    for i in range(batch_n):
        batch_x=data_x[i*batch_size:(i+1)*batch_size]
        batch_y=data_y[i*batch_size:(i+1)*batch_size]
        
        yield batch_x,batch_y

In [358]:
epochs = 200
train_collect = 50
train_print=train_collect*10

learning_rate_value = 0.001
batch_size=32


x_collect = []
train_loss_collect = []
train_acc_collect = []
valid_loss_collect = []
valid_acc_collect = []

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    iteration=0
    for e in range(epochs):
        for batch_x,batch_y in get_batch(train_x,train_y,batch_size):
            iteration+=1
            feed = {model.inputs: train_x,
                    model.labels: train_y,
                    model.learning_rate: learning_rate_value,
                    model.is_training:True
                   }

            train_loss, _, train_acc = sess.run([model.cost, model.optimizer, model.accuracy], feed_dict=feed)
            
            if iteration % train_collect == 0:
                x_collect.append(e)
                train_loss_collect.append(train_loss)
                train_acc_collect.append(train_acc)

                if iteration % train_print==0:
                     print("Epoch: {}/{}".format(e + 1, epochs),
                      "Train Loss: {:.4f}".format(train_loss),
                      "Train Acc: {:.4f}".format(train_acc))
                        
                feed = {model.inputs: valid_x,
                        model.labels: valid_y,
                        model.is_training:False
                       }
                val_loss, val_acc = sess.run([model.cost, model.accuracy], feed_dict=feed)
                valid_loss_collect.append(val_loss)
                valid_acc_collect.append(val_acc)
                
                if iteration % train_print==0:
                    print("Epoch: {}/{}".format(e + 1, epochs),
                      "Validation Loss: {:.4f}".format(val_loss),
                      "Validation Acc: {:.4f}".format(val_acc))
                

    saver.save(sess, "./titanic.ckpt")

Epoch: 23/200 Train Loss: 0.4299 Train Acc: 0.8048
Epoch: 23/200 Validation Loss: 0.4473 Validation Acc: 0.8380
Epoch: 46/200 Train Loss: 0.4079 Train Acc: 0.8230
Epoch: 46/200 Validation Loss: 0.3479 Validation Acc: 0.8771
Epoch: 69/200 Train Loss: 0.4015 Train Acc: 0.8301
Epoch: 69/200 Validation Loss: 0.3319 Validation Acc: 0.8715
Epoch: 91/200 Train Loss: 0.3964 Train Acc: 0.8287
Epoch: 91/200 Validation Loss: 0.3322 Validation Acc: 0.8827
Epoch: 114/200 Train Loss: 0.3919 Train Acc: 0.8357
Epoch: 114/200 Validation Loss: 0.3265 Validation Acc: 0.8827
Epoch: 137/200 Train Loss: 0.3869 Train Acc: 0.8385
Epoch: 137/200 Validation Loss: 0.3272 Validation Acc: 0.8547
Epoch: 160/200 Train Loss: 0.3804 Train Acc: 0.8343
Epoch: 160/200 Validation Loss: 0.3224 Validation Acc: 0.8883
Epoch: 182/200 Train Loss: 0.3768 Train Acc: 0.8371
Epoch: 182/200 Validation Loss: 0.3191 Validation Acc: 0.8939


In [359]:
model=build_neural_network()
restorer=tf.train.Saver()
with tf.Session() as sess:
    restorer.restore(sess,"./titanic.ckpt")
    feed={
        model.inputs:X_test,
        model.is_training:False
    }
    test_predict=sess.run(model.predicted,feed_dict=feed)
    
test_predict[:10]

INFO:tensorflow:Restoring parameters from ./titanic.ckpt


array([[ 0.09059235],
       [ 0.81261176],
       [ 0.07943363],
       [ 0.08803612],
       [ 0.45018101],
       [ 0.19406626],
       [ 0.60541189],
       [ 0.31591845],
       [ 0.61372489],
       [ 0.08567898]], dtype=float32)

In [360]:
from sklearn.preprocessing import Binarizer
binarizer=Binarizer(0.5)
test_predict_result=binarizer.fit_transform(test_predict)
test_predict_result=test_predict_result.astype(np.int32)
test_predict_result[:10]

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0]], dtype=int32)

In [361]:
passenger_id=test_passenger_id.copy()
evaluation=passenger_id.to_frame()
evaluation["Survived"]=test_predict_result
evaluation[:10]

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [362]:
evaluation.to_csv("nn_submission_v1.csv",index=False)